In [2]:
import pandas as pd

In [3]:
t = ['Equal or Less than Limit of Detection (LOD = 0.03%) ',
' 0.09% ', ' 0.09%',
' The retention time of the peak due to Domperidone in the chromatogram of the sample preparation.',
     0.12,230,12,1.12
]

In [4]:
df = pd.DataFrame(t,columns =["val"]) 
df

,val
0,Equal or Less than Limit of Detection (LOD = 0...
1,0.09%
2,0.09%
3,The retention time of the peak due to Domperi...
4,0.12
5,230
6,12
7,1.12


In [6]:
df["New"] = pd.to_numeric(df["val"], errors='coerce')
df

,val,New
0,Equal or Less than Limit of Detection (LOD = 0...,NaN
1,0.09%,NaN
2,0.09%,NaN
3,The retention time of the peak due to Domperi...,NaN
4,0.12,0.12
5,230,230.00
6,12,12.00
7,1.12,1.12


#### Finding the median and quantile values of a group

In [22]:
t = {
    'val' : [12,11, 12,123,0.12,230,12,1.12],
'category' : ['a','b','c','a','b','c','a','b']
}

In [23]:
df = pd.DataFrame(t) 
df

,val,category
0,12.00,a
1,11.00,b
2,12.00,c
3,123.00,a
4,0.12,b
5,230.00,c
6,12.00,a
7,1.12,b


In [24]:
df.groupby('category')['val'].median()

category
a     12.00
b      1.12
c    121.00
Name: val, dtype: float64

In [28]:
df.groupby('category').agg({'val':['median','sum','first','last','count']})

val                            
          median     sum first    last count
category                                    
a          12.00  147.00  12.0   12.00     3
b           1.12   12.24  11.0    1.12     3
c         121.00  242.00  12.0  230.00     2

In [29]:
def qtu(x):
    return x.quantile(0.75)

def qtl(x):
    return x.quantile(0.25)

In [30]:
df.groupby('category').agg({'val':'median'})

,val
category,
a,12.00
b,1.12
c,121.00


In [31]:
df.groupby('category').agg({'val':['median',qtu,qtl]})

val               
          median     qtu    qtl
category                       
a          12.00   67.50  12.00
b           1.12    6.06   0.62
c         121.00  175.50  66.50

In [32]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn. ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [33]:
data = pd.read_csv('mnist_train.csv')

In [34]:
data.shape

(60000, 785)

In [35]:
data

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
df_x = data.iloc[:,1:]
df_y = data.iloc[:,0]

In [37]:
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size = 0.2, random_state = 4)

In [38]:
dt = DecisionTreeClassifier()
dt.fit(x_train,y_train)

DecisionTreeClassifier()

In [39]:
dt.score(x_train,y_train)

1.0

In [40]:
dt.score(x_test,y_test)

0.8643333333333333

In [41]:
bg = BaggingClassifier(dt, n_estimators = 5)

In [42]:
bg.fit(x_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=5)

In [43]:
bg.score(x_test,y_test)

0.921

In [44]:
bg.score(x_train,y_train)

0.9934375

In [45]:
rf = RandomForestClassifier(n_estimators = 5)
rf.fit(x_train,y_train)

RandomForestClassifier(n_estimators=5)

In [21]:
rf.score(x_test,y_test)

0.9165

In [47]:
lr = LogisticRegression()

svm = SVC(kernel = 'poly', degree = 2)

In [23]:
evc = VotingClassifier(estimators = [('lr',lr),('dt',dt),('svm',svm)],voting = 'hard')

In [48]:
evc.fit(x_train.iloc[1:400],y_train.iloc[1:400])

NameError: name 'evc' is not defined

In [19]:
evc.score(x_test,y_test)

0.80775